Notes from AI Labs
Split Data before permutation importance - 70/30 split.  Can do 5 different splits

When evaluating - need evaluation matrix, true positives and true negatives with false positives and false negatives

Precision = True Positives/(True Positives + False Positives)

Accuracy = True Positives and Negatives/(all four categories)

True Positive Rate, Recall = TP/(TP+FN)

False Positive Rate, FP/(FP+TN)

C-index(Concordance Index) in Survival Analysis

Measures the agreement between predicted risk scores and observed survival outcomes

C-index = Number of concordant patient pairs/(Total comparable patient pairs)

Goal of project is to maximize C-index

Survival Analysis when it's binary can be random forest

In [1]:
#import kagglehub - no longer needed once using local .csv file
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, ShuffleSplit
#from sklearn.preprocessing import LabelEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.preprocessing import StandardScaler
# Define a scorer compatible with GridSearchCV
from sklearn.metrics import make_scorer

## Skip below after first run ##

In [ ]:
path = kagglehub.dataset_download("reihanenamdari/breast-cancer")
path #path to data download on local machine

100%|██████████| 42.8k/42.8k [00:00<00:00, 1.22MB/s]

Extracting files...


In [2]:
df = pd.read_csv('Breast_Cancer.csv')
#4000 total records

In [3]:
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [4]:
#Create numerical columns out of Race
df['White'] = (df['Race']=='White').astype(int)
df['Black'] = (df['Race']=='Black').astype(int)
df['Other'] = (df['Race']=='Other').astype(int)

In [5]:
#Drop categorical columns - drops Race, Marital Status, 6th stage, differentiate, A stage
df.drop(df.columns[[1,2,5,6,8]], axis=1, inplace=True)

In [6]:
#Encode Categorical Columns
df['T Stage '] = df['T Stage '].map({'T1':1,'T2':2, 'T3':3,'T4':4})
df['N Stage'] = df['N Stage'].map({'N1':1,'N2':2, 'N3':3})
df['Estrogen Status'] = df['Estrogen Status'].map({'Positive':1,'Negative':0})
df['Progesterone Status'] = df['Progesterone Status'].map({'Positive': 1,'Negative': 0})
df['Status'] = df['Status'].map({'Alive':1,'Dead':0})
#Force to numeric and drop those with missing grades
df['Grade'] = pd.to_numeric(df['Grade'], errors = 'coerce')
df = df.dropna(subset = ['Grade'])

In [7]:
#check shape of dataset
print(df.shape)

(4005, 14)


In [8]:
#Create a survival object dataframe
y = df[['Status','Survival Months']]

In [9]:
y.head()

,Status,Survival Months
0,1,60
1,1,62
2,1,75
3,1,84
4,1,50


Below will create a list of tuples, 'Status' and 'Survival Months' by row and keeps Status boolean
and Survival Months as a 64 bit float

These two variables define survival time and status as target variables.

This helps us use sckit-surival library for analysis.  It requires this tuple format.S

In [10]:
y_structured = np.array([(bool(status), months) for status, months in zip(y['Status'], y['Survival Months'])],
                        dtype = [('Status','bool'), ('Survival Months', 'f8')])

In [11]:
y_structured 
# Status of True means event happened (Death)

array([( True,  60.), ( True,  62.), ( True,  75.), ..., ( True,  69.),
       ( True,  72.), ( True, 100.)],
      shape=(4005,), dtype=[('Status', '?'), ('Survival Months', '<f8')])

In [12]:
#Removes target columns from original dataframe, which contains our features only
X= df.drop(columns = ['Status','Survival Months'])

In [13]:
#X becomes our Independent variables
X.head()

,Age,T Stage,N Stage,Grade,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,White,Black,Other
0,68,1,1,3.0,4,1,1,24,1,1,0,0
1,50,2,2,2.0,35,1,1,14,5,1,0,0
2,58,3,3,2.0,63,1,1,14,7,1,0,0
3,58,1,1,3.0,18,1,1,2,1,1,0,0
4,47,2,1,3.0,41,1,1,3,1,1,0,0


In [14]:
# --- Split data into training, validation, and test sets ---
# First, split into training+validation and test sets (e.g., 80% train+val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_structured, test_size=0.2, random_state=42)

# Then, split the training+validation set into training and validation sets (e.g., 60% train, 20% validation of the original data)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=(0.25), random_state=42)
# Note: 0.25 of 0.8 is 0.2, so the final split is 60% train, 20% validation, 20% test

print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of validation samples: {X_val.shape[0]}")
print(f"Number of test samples: {X_test.shape[0]}")

Number of training samples: 2403
Number of validation samples: 801
Number of test samples: 801


In [ ]:
# --- Feature Scaling (StandardScaler) ---
#Goal is to scale the numerical features in training, test and validation data sets to prevent 'leakage' from validation and test sets into training
#Important because it puts all features on a comparable scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [16]:
# Custom scoring function for survival analysis - closer to 1 is better.  0.5 is a random guess.  
# C-index measures how well the model predicts who survives longer
# Custom scoring function for survival analysis
def cindex_score(model, x, y_struct):
    prediction = model.predict(x)
    return concordance_index_censored(y_struct['Status'], y_struct['Survival Months'], prediction)[0]

**Training and Test Split - with 5 splits**

Each split gets a Random Survival forest Model and evaluates each model using cindex_score, while storing and averaging the results.

In [17]:
print("\n--- Cross-Validation C-Indices (on training data) ---")
rs = ShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
split_scores = []

#Here we are training multiple models
for train_idx, val_idx in rs.split(X_train_scaled):  # Splitting the scaled training data for cross-validation
    X_train_fold, X_val_fold = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    model = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42, n_jobs=-1)
    model.fit(X_train_fold, y_train_fold)

    c_index = cindex_score(model, X_val_fold,y_val_fold)
    split_scores.append(c_index)
    print(f"C-Index for one split: {c_index:.4f}")

print(f"\nAverage C-Index across 5 splits (on training data): {np.mean(split_scores):.4f}")


--- Cross-Validation C-Indices (on training data) ---
C-Index for one split: 0.4983
C-Index for one split: 0.4942
C-Index for one split: 0.5012
C-Index for one split: 0.5134
C-Index for one split: 0.4952

Average C-Index across 5 splits (on training data): 0.5005


Collect all models and test sets and do permutation importance for each feature then average.

We shuffle the values, and model performance drop after shuffling indicates how important that feature is.

We repeat 10x for each feature and we average this across the 5 CV splits.

**Ultimately this helps us determine which features actually contribute to model performance**

In [ ]:
# Calculate average permutation importance across all splits
importances_list = []

for train_idx, val_idx in rs.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    model = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42, n_jobs=-1)
    model.fit(X_train_fold, y_train_fold)

    perm_result = permutation_importance(
        estimator=model,
        X=X_val_fold,  # Using validation fold for importance
        y=y_val_fold,  # Using validation fold for importance
        n_repeats=10,
        random_state=42,
        n_jobs=-1,
        scoring=cindex_score
    )
    importances_list.append(perm_result.importances_mean)

# Average the importances
avg_importances = np.mean(importances_list, axis=0)
importance_df = pd.Series(avg_importances, index=X.columns).sort_values(ascending=False)

print("\nPermutation importances (averaged across splits on training data):\n", importance_df)

#Positive values means model performance decrease, negative actually causing a model performance to increase
#This means positive values are strongly predictive of survival outcomes, near 0 are not contributing to predictions and negative values may be confusing model



Permutation importances (averaged across splits on training data):
 Estrogen Status           0.002197
Reginol Node Positive     0.002139
T Stage                   0.002121
Grade                     0.001768
White                     0.000895
Other                     0.000659
N Stage                  -0.000516
Black                    -0.001442
Progesterone Status      -0.001601
Tumor Size               -0.003243
Age                      -0.004102
Regional Node Examined   -0.005902
dtype: float64


**Feature and Model Selection**

We do permutation based feature importance to identify top X features.  This reduces dimensionality and focuses on variables we think are more predictive.

For each fold we train a full model, identify the top k most important features, train another model using only those k features and then evaluate performance.

Random Survival Forest is chosen as the model approach due to its robustness with non-linear relationships and handling high dimensional data.  Random Survival Forest is good for predicting survival time distributions.

In [19]:
print("\n--- Cross-Validated Top-K Features Testing (on training data) ---")

rs_top_k = ShuffleSplit(n_splits=5, test_size=0.25, random_state=42)

importances_list = []   
all_k_scores = {}       
best_score = 0
best_k = 0

for k in range(3, 11):
    cindex_scores_k = []

    for train_idx, val_idx in rs_top_k.split(X_train_scaled):
        X_train_fold, X_val_fold = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Step 1: Train full model on training fold
        model = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42, n_jobs=-1)
        model.fit(X_train_fold, y_train_fold)

        # Step 2: Permutation importance on validation fold
        perm_result = permutation_importance(
            estimator=model,
            X=X_val_fold,
            y=y_val_fold,
            n_repeats=10,
            random_state=42,
            n_jobs=-1,
            scoring=cindex_score
        )
        importances_list.append(perm_result.importances_mean)

        # Step 3: Select top K features from the entire scaled training set based on averaged importance
        importance_train = pd.Series(np.mean(importances_list, axis=0), index=X_train_scaled.columns)
        top_k_features = importance_train.sort_values(ascending=False).head(k).index

        # Step 4: Train new model on only Top-K features on the training fold
        model_k = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42, n_jobs=-1)
        model_k.fit(X_train_fold[top_k_features], y_train_fold)

        # Step 5: Evaluate C-index on validation fold
        c_index_k = cindex_score(model_k, X_val_fold[top_k_features], y_val_fold)
        cindex_scores_k.append(c_index_k)

    avg_cindex = np.mean(cindex_scores_k)
    all_k_scores[k] = avg_cindex

    print(f"Top {k} features average C-Index across splits (on training data): {avg_cindex:.4f}")

    if avg_cindex > best_score:
        best_score = avg_cindex
        best_k = k
print(f"\nBest K based on C-index (on training data): {best_k} with C-Index: {best_score:.4f}")

# Average the collected permutation importances across all runs
avg_importances = np.mean(importances_list, axis=0)

# Build importance DataFrame and select final features based on the entire scaled training set
importance_df = pd.Series(avg_importances, index=X_train_scaled.columns).sort_values(ascending=False)
top_features = importance_df.head(best_k).index

# Final X_selected for hyperparameter tuning (using the best K features from the scaled training data)
X_selected = X_train_scaled[top_features]


--- Cross-Validated Top-K Features Testing (on training data) ---
Top 3 features average C-Index across splits (on training data): 0.5161
Top 4 features average C-Index across splits (on training data): 0.5168
Top 5 features average C-Index across splits (on training data): 0.5085
Top 6 features average C-Index across splits (on training data): 0.5085
Top 7 features average C-Index across splits (on training data): 0.5092
Top 8 features average C-Index across splits (on training data): 0.5095
Top 9 features average C-Index across splits (on training data): 0.5059
Top 10 features average C-Index across splits (on training data): 0.5075

Best K based on C-index (on training data): 4 with C-Index: 0.5168


**Hyperparameter Tuning**

Simply, we look at all combinations of trees and number of features selected at each split.  This aims to optimize Concordance Index, which is a standard metric in survival analysis.

In [20]:
def concordance_index_scorer(estimator, X, y):
    return concordance_index_censored(y['Status'], y['Survival Months'], estimator.predict(X))[0]

param_grid = {
    'n_estimators': [50,100,150,200,250,300,350,400,450,500],
    'max_features': [1, 2, 3, 4, 5,'sqrt'],
    'min_samples_split': [3, 5,7,10,13,15],
    'min_samples_leaf': [3,5,7,10,13,15]
}

grid_search = GridSearchCV(
    estimator=RandomSurvivalForest(random_state=42),
    param_grid=param_grid,
    scoring=make_scorer(concordance_index_scorer),
    cv=3, #Using folds on the scaled training data
    n_jobs=-1
)

grid_search.fit(X_selected, y_train)  # Fitting on the scaled training data with selected features
print("\nBest parameters from GridSearchCV:", grid_search.best_params_)

c:\Python\BreastCancerPrognosis\AICampus\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
1080 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python\BreastCancerPrognosis\AICampus\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python\BreastCancerPrognosis\AICampus\Lib\site-packages\sksurv\ensemble\forest.py", line 183, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, prefer="threads")(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Best parameters from GridSearchCV: {'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 50}


In [21]:
# Train best model on the entire scaled training set with the selected features and best hyperparameters
best_rsf = grid_search.best_estimator_
best_rsf.fit(X_train_scaled[top_features], y_train)

# Evaluate the best model on the scaled validation set
val_c_index = concordance_index_censored(
    event_indicator=y_val['Status'],
    event_time=y_val['Survival Months'],
    estimate=best_rsf.predict(X_val_scaled[top_features])
)[0]

print(f'\nValidation Concordance Index: {val_c_index:.4f}')

# --- Final Evaluation on the Scaled Test Set ---
final_c_index_test = concordance_index_censored(
    event_indicator=y_test['Status'],
    event_time=y_test['Survival Months'],
    estimate=best_rsf.predict(X_test_scaled[top_features])
)[0]

print(f'\nFinal Concordance Index on Test Set: {final_c_index_test:.4f}')


Validation Concordance Index: 0.5009

Final Concordance Index on Test Set: 0.5205


**Result interpretation**

0.5: Model performs no better than random chance

Greater than 0.7: Indicates good discriminative ability

Greater than 0.8: Indicates strong predictive performance.  A higher C-index suggests that the model effectively distinguishes between patients with different survival outcomes

In [22]:
# --- Evaluation Matrix (at a specific time point) on the Test Set ---
time_horizon = 60  # Example: 60 months

# Convert test set survival data to binary based on time horizon
y_test_binary = np.array([1 if months > time_horizon and not status else 0 for status, months in y_test])

# Get risk scores from your best model on the scaled test set
risk_predictions_test = best_rsf.predict(X_test_scaled[top_features])

# You'll need to determine a threshold on the risk scores to convert to binary predictions
# This is a crucial step and can influence your results.
# For demonstration, let's use the median risk on the validation set as a threshold
risk_predictions_val = best_rsf.predict(X_val_scaled[top_features])
threshold = np.median(risk_predictions_val)
binary_predictions_test = (risk_predictions_test < threshold).astype(int) # Higher risk might mean lower survival probability

# Calculate confusion matrix
cm = confusion_matrix(y_test_binary, binary_predictions_test)
print("\nConfusion Matrix (at {} months on Test Set):\n".format(time_horizon), cm)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = cm.ravel()
print("\nTrue Positives (Survived beyond {} months):".format(time_horizon), TP)
print("True Negatives (Died within {} months):".format(time_horizon), TN)
print("False Positives (Predicted survived, actually died):", FP)
print("False Negatives (Predicted died, actually survived):", FN)


Confusion Matrix (at 60 months on Test Set):
 [[445 322]
 [ 18  16]]

True Positives (Survived beyond 60 months): 16
True Negatives (Died within 60 months): 445
False Positives (Predicted survived, actually died): 322
False Negatives (Predicted died, actually survived): 18
